In [7]:
import os
import requests
import operator
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.callbacks import get_openai_callback

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_PROJECT"] = "RRL project"

# Just fill these 2 and you should be good 
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""


### TODO:
- Add multihop question generation.
- Add features to solve failcase 2 and 3

In [21]:
def call_orchestrator(messages, model):
    "Takes in user query -> returns relevant tool to call from the list of available tools."
    
    template = """You are Hotel booking platform orchestrator who can use tools to retrieve relevant information.
        Your objective it to answer user query in the most optimal way and so you must use the tool available to
        to do so.
        
        Tools available: ['check_query', 'search_web', 'generate_answer', 'done']
        check_query: to check if most of the information to suggest a hotel is present in the query.
        search_web: to search the web with the user query for relevant hotels.
        generate_answer: to answer user query based on web search results.
        done: to break the workflow when the user query has been successfully answered.
        
        Return type: str
        return only the name of the tool to be used.
        
        History: {messages}"""
    
    print(messages)
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "messages": str(messages),
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Orchestrator output: " + response.content)
    return response.content, cb.total_cost

def check_query(messages, model):
    """Takes in user query and checks if it has all relevant information to book hotel.
        yes -> return to the orchestrator. 
        no -> return to the user."""
    
    print(messages)
    template = """You are a query checker for hotel room booking agent. Your job is to verify if the given user query consist of all relevant information
        to proceed with suggesting a hotel for the user. The query must contain information such as number of days of stay, number of rooms, number of guests etc.
        
        Query: {messages}
        
        Return type: str
        return only True or False where True means the query is sufficient and False represents it is not."""
    
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "messages": str(messages[-2]),
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Query check: " + response.content)
    return response.content, cb.total_cost


def get_updated_query(messages):
    "Ask the user to share more information about their booking choice."

    print(messages)
    updated_query = input("The entered information is not sufficient to search for hotels, please share more information like location, dates, number of guests, budget etc.")
    messages.append(updated_query)
    return updated_query

def search(messages):
    "Takes in user query -> searches the web -> return relevant information."
    print(messages)
    search_query = messages[-4]

    def fetch_full_text(url):
        try:
            response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
            response.raise_for_status()  # Raise an error for bad responses
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Extract main text content from <p> tags
            paragraphs = soup.find_all("p")
            full_text = "\n".join([p.get_text() for p in paragraphs])
            return full_text
        except requests.exceptions.RequestException as e:
            return f"Error fetching page: {e}"

    # Function to search DuckDuckGo
    def search_duckduckgo(query, num_results=3):
        searched_output = []
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=num_results))
        for i, search_results in enumerate(results, 1):
            link_text = fetch_full_text(search_results['href'])
            result = f"\nSearch: {i}\nTitle: {search_results['title']}\nBody: {link_text}"
            # print(result)
            searched_output.append(result)
        return searched_output
    
    search_results = search_duckduckgo(search_query, num_results=5)

    return search_results

def generate_answer(messages, query):
    "Based on the obtained search results answer the user query"

    print(messages)
    web_search = messages[-2]

    template = """You are a hotel booking platform QA bot employed to answer user query based on retrieved information. Do not use your own knowledge
        but rely only on the extracted information.
        
        Web search: {web_search}
        
        Query: {query}"""
    
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "web_search": str(web_search),
                "query": str(query)
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Answer: " + response.content)
    return response.content, cb.total_cost

def done(messages):
    "Used as an indicator to terminate the program."
    
    if "done" in messages[-1]:
        return True


In [ ]:
completed_task = False
messages = []
query = input("Hi, how can I assist you.") # hotels with 3 rooms 2 bathroom for 3 people for 3 nights
messages.append(query)

model = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21",  temperature=0, max_tokens=512, timeout=None, max_retries=2)

while not completed_task:
    choice_tool, _ = call_orchestrator(messages, model)

    if choice_tool == "check_query":
        flag, _ = check_query(messages, model)
        if flag == "False":
            flag = False
        elif flag == "True":
            flag = True
        if not flag:
            query = get_updated_query(messages)
    elif choice_tool == "search_web":
        web_results = search(messages)
        messages.append(web_results)
        print(messages)
    elif choice_tool == "generate_answer":
        response = generate_answer(messages, query)
    elif choice_tool == "done":
        print("Inside ELIF")
        completed_task = done(messages)
        # print(completed_task)

['hotels with 3 rooms 2 bathroom for 3 people for 3 nights']
Total Cost (USD): $0.000470
['hotels with 3 rooms 2 bathroom for 3 people for 3 nights', 'Orchestrator output: check_query']
Total Cost (USD): $0.000295
['hotels with 3 rooms 2 bathroom for 3 people for 3 nights', 'Orchestrator output: check_query', 'Query check: True']
Total Cost (USD): $0.000508
['hotels with 3 rooms 2 bathroom for 3 people for 3 nights', 'Orchestrator output: check_query', 'Query check: True', 'Orchestrator output: search_web']
['hotels with 3 rooms 2 bathroom for 3 people for 3 nights', 'Orchestrator output: check_query', 'Query check: True', 'Orchestrator output: search_web', ['\nSearch: 1\nTitle: Can 3 People Stay In A Hotel Room? A Comprehensive Guide\nBody: No products in the cart.\nHome - Hotel Guide - Can 3 People Stay In A Hotel Room? A Comprehensive Guide\nTraveling with friends or family can be an exciting adventure, but it often comes with the challenge of finding affordable accommodation. One q

In [27]:

print(messages[-2])

Answer: Based on the retrieved information, here are some options for hotels that offer accommodations with 3 rooms and 2 bathrooms for 3 people for 3 nights:

1. **Staybridge Suites Tysons - McLean, Virginia (IHG Hotel)**:
   - Offers three-bedroom suites that can accommodate up to 8 guests.
   - Includes full kitchens, complimentary hot breakfast, outdoor pool, grilling area, and fitness center.
   - Conveniently located near Washington, D.C., and major airports.

2. **WhyHotel Tysons Corner Greensboro Drive, McLean, Virginia**:
   - Offers a three-bedroom/two-bath suite option, as well as a three-bedroom apartment.
   - Provides easy access to nearby attractions and amenities.

3. **Homewood Suites by Hilton Hagerstown, Maryland**:
   - Offers a 3-bedroom/3-bath suite.
   - Located near Valley Mall and local attractions like the Chesapeake & Ohio National Historical Park.
   - Includes amenities such as a 24-hour fitness center, indoor pool, and business center.

4. **Four Seasons D